In [1]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

In [2]:
path='../../data/data2/raw_2/deliveries.csv'

In [3]:
deliveries = pd.read_csv(path,encoding='latin-1')

In [4]:
date_fields = [
    "pickup_dt", "max_pickup_dt", "pickup_effective_dt",
    "delivery_dt", "max_delivery_dt", "delivery_effective_dt"
]
for field in date_fields:
    deliveries = deliveries.loc[deliveries[field] != "0000-00-00 00:00:00"]
    deliveries[field] = pd.to_datetime(deliveries[field])
deliveries.dropna(subset=date_fields, inplace=True)
deliveries.shape


(594268, 26)

In [5]:
path_steps='../../data/data2/raw_2/steps-001.csv'

In [6]:
steps = pd.read_csv(path_steps)

In [7]:
steps.drop_duplicates(inplace=True)

In [8]:
group_routes=steps.groupby('route_id')

In [9]:
group_route_delivery=deliveries.groupby('route_id')

In [10]:
size_complete_routes=group_routes.size()
routes_to_keep=[]
routes_not_keep=[]
for route_id, size_r in group_route_delivery.size().iteritems():
    if size_complete_routes.loc[route_id]//2 == size_r:
        routes_to_keep.append(route_id)
    else:
        routes_not_keep.append(route_id)

In [11]:
group_routes_to_keep=group_routes.filter(lambda x: x.name in routes_to_keep)
group_route_delivery_to_keep=group_route_delivery.filter(lambda x: x.name in routes_to_keep)

In [12]:
steps=group_routes_to_keep.reset_index(drop=True)
deliveries=group_route_delivery_to_keep.reset_index(drop=True)

In [13]:
steps= steps.merge(deliveries,left_on='delivery_id', right_on='id', how='left')

In [14]:
#show all columns on just one page
pd.set_option('display.max_columns', None)

In [15]:
columns_to_drop=['route_id_y','state','_status','items','id','max_pickup_dt','pickup_arrival_dt','pickup_dt','delivery_dt','max_delivery_dt','delivery_arrival_dt','total_package_price']

In [16]:
steps.drop(columns_to_drop, axis=1,inplace=True)

In [17]:
steps.sort_values(by=['route_id_x','step_order'],inplace=True)

In [18]:
steps['effective_dt']=steps.apply(lambda x: x['pickup_effective_dt'] if x['action']=='pickup' else x['delivery_effective_dt'],axis=1)

In [19]:
steps.drop(columns=['pickup_effective_dt','delivery_effective_dt'],inplace=True)

In [20]:
steps.reset_index(drop=True,inplace=True)

In [21]:
def get_delta_time(row, final_index):
    index = row.name
    is_last_row = index == final_index
    delta_time = 0
    if not is_last_row:
        delta_time = (steps.loc[index + 1, 'effective_dt'] -
                      steps.loc[index, 'effective_dt']).total_seconds()
    return delta_time


def get_previous_step_data(step):
    index = step.name
    is_first_step = step.step_order == 1
    time_from_previous = 0
    if not is_first_step:
        time_from_previous = (
            step.effective_dt - steps.loc[index - 1, 'effective_dt']).total_seconds()
    return time_from_previous


In [22]:
index_last_row=steps.shape[0]-1

In [23]:
index_last_row

592841

In [24]:
steps['time_to_next']=steps.apply(lambda row: get_delta_time(row,index_last_row), axis=1)
steps['time_from_previous']=steps.apply(lambda row: get_previous_step_data(row), axis=1)

In [35]:
steps.columns

Index(['route_id_x', 'delivery_id', 'action', 'step_order', 'distance_x',
       'total_size', 'pickupLat', 'pickupLng', 'deliveryLat', 'deliveryLng',
       'cost', 'final_price', 'distance_y', 'shipper_id', 'distance_charged',
       'distance_route', 'vehicle_capacity', 'effective_dt', 'time_to_next',
       'time_from_previous'],
      dtype='object')

In [36]:
steps_to_predict=steps[steps['time_to_next']<0][['route_id_x','delivery_id','distance_x']]

In [32]:
import pickle

In [33]:
loaded_model = pickle.load(open('./model.pickle', 'rb'))

In [38]:
result = loaded_model.predict(steps_to_predict[['distance_x']])

In [40]:
steps.loc[steps['time_to_next']<0,'time_to_next']=result

In [ ]:
steps.to_csv('../../data/data2/clean_2/steps_predicted.csv', index=False)

In [25]:
time_to_next_percentage=steps.groupby('route_id_x').agg({'time_to_next': lambda x: (x>0).sum()/x.count() }).reset_index()

In [26]:
(time_to_next_percentage.time_to_next>=0).sum()

9730

In [87]:
good_steeps=steps[(steps['time_to_next']>=0) & (steps['distance_x']!=0)][['distance_x','time_from_previous']]

In [91]:
good_steeps.to_csv('../../data/data2/clean_2/good_steps.csv', index=False)

In [69]:
steps[steps.route_id_x=='r-shiinc-1011382']

,route_id_x,delivery_id,action,step_order,distance_x,total_size,pickupLat,pickupLng,deliveryLat,deliveryLng,cost,final_price,distance_y,shipper_id,distance_charged,distance_route,vehicle_capacity,effective_dt,time_to_next,time_from_previous
0,r-shiinc-1011382,t-petz-2548122,pickup,1,0.0,1,-19.9627,-43.957,-19.9836,-43.9721,6.48,10.0,5.2,170162.0,2.9,2.9,1,2022-01-03 12:30:51,0.0,0.0
1,r-shiinc-1011382,t-petz-2547366,pickup,2,0.0,4,-19.9627,-43.957,-19.9758,-43.9748,6.48,10.0,3.8,170162.0,1.0,1.0,4,2022-01-03 12:30:51,0.0,0.0
2,r-shiinc-1011382,t-petz-2546288,pickup,3,0.0,3,-19.9627,-43.957,-19.9725,-43.9703,6.48,10.0,3.2,170162.0,1.1,1.1,3,2022-01-03 12:30:51,0.0,0.0
3,r-shiinc-1011382,t-petz-2546397,pickup,4,0.0,1,-19.9627,-43.957,-19.9650,-43.9651,6.48,10.0,2.3,170162.0,2.0,2.0,1,2022-01-03 12:30:51,0.0,0.0
4,r-shiinc-1011382,t-petz-2547114,pickup,5,0.0,3,-19.9627,-43.957,-19.9791,-43.9741,6.48,10.0,4.2,170162.0,1.1,1.1,3,2022-01-03 12:30:51,0.0,0.0
5,r-shiinc-1011382,t-petz-2547535,pickup,6,0.0,2,-19.9627,-43.957,-19.9473,-43.9719,6.48,10.0,4.6,170162.0,2.5,2.5,2,2022-01-03 12:30:51,0.0,0.0
6,r-shiinc-1011382,t-petz-2547505,pickup,7,0.0,1,-19.9627,-43.957,-19.9754,-43.9418,6.48,10.0,3.1,170162.0,1.7,1.7,1,2022-01-03 12:30:51,0.0,0.0
7,r-shiinc-1011382,t-petz-2546920,pickup,8,0.0,1,-19.9627,-43.957,-19.9743,-43.9729,6.48,10.0,3.7,170162.0,0.5,0.5,1,2022-01-03 12:30:51,0.0,0.0
8,r-shiinc-1011382,t-petz-2547538,pickup,9,0.0,3,-19.9627,-43.957,-19.9691,-43.9502,6.48,10.0,3.4,170162.0,3.4,3.4,3,2022-01-03 12:30:51,0.0,0.0
9,r-shiinc-1011382,t-petz-2546919,pickup,10,0.0,5,-19.9627,-43.957,-19.9743,-43.9729,6.48,10.0,3.7,170162.0,0.0,0.0,5,2022-01-03 12:30:51,0.0,0.0
